# First test

In this file, we do not apply any transformations to the data. We only rescale images and pass them through a pretrained CNN model.
The objective is to determine whether a pretrained model is already capable of finding relevant information in these images, without transforming them, and in 224x224.

## Import Dataset from Kaggle

In [2]:
! pip install -q kaggle

In [3]:
from google.colab import files

In [ ]:
files.upload()

In [6]:
! mkdir ~/.kaggle

In [7]:
! cp kaggle.json ~/.kaggle/

In [8]:
! chmod 600 ~/.kaggle/kaggle.json

In [9]:
! kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

100% 2.29G/2.29G [00:55<00:00, 45.3MB/s]
100% 2.29G/2.29G [00:55<00:00, 44.1MB/s]


In [ ]:
! unzip chest-xray-pneumonia.zip -d data/

## Load model

In [1]:
from torchvision.models import resnet34, ResNet34_Weights
import torch

In [2]:
pretrained = resnet34(weights=ResNet34_Weights.IMAGENET1K_V1)

In [3]:
pretrained.fc = torch.nn.Identity()
pretrained.avgpool = torch.nn.Identity()

In [4]:
class Model(torch.nn.Module):
    def __init__(self, pretrained):
        super(Model, self).__init__()
        self.pretrained = pretrained
        self.linear1 = torch.nn.Linear(25088, 10000)
        self.linear2 = torch.nn.Linear(10000, 1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.pretrained(x)
        x = self.linear1(x)
        x = self.linear2(x)
        x = self.sigmoid(x)
        return x

In [5]:
model = Model(pretrained)

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
model = model.to(device)

## Load Data and Preprocess


In this test, the only preprocessing applied is a basic resize to (224, 224, 3), and a normalization using Pytorch Transforms.  
We do not modify the images. Aspect ratio is taken into account during the resize process.

In [8]:
import math
import os
import cv2
import torchvision.transforms as transforms
from Dataset import Dataset
from data_loading import open_preprocess_photos

In [9]:
normal_dir: str = r'./data/chest_xray/test/NORMAL'
pneumo_dir: str = r'./data/chest_xray/test/PNEUMONIA'

assert os.path.exists(normal_dir) and os.path.isdir(normal_dir), "Normal dir isn't found or isn't a directory"
assert os.path.exists(pneumo_dir) and os.path.isdir(pneumo_dir), "Pneumonia dir isn't found or isn't a directory"

In [10]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

transform = transforms.Compose([
    transforms.ToTensor(),
    normalize,
])

In [11]:
normal = open_preprocess_photos(normal_dir, transform, (224, 224))
pneumonia = open_preprocess_photos(pneumo_dir, transform, (224, 224))

In [12]:
dataset = Dataset(normal, pneumonia, 0, 1, 128)

In [13]:
dataset[0][0].shape  # Should be as big as the given batch_size, so 128

torch.Size([128, 3, 224, 224])

## Training

In [14]:
lr=1e-2
momentum=0.9
batch_size=1024

optimizer = torch.optim.SGD(model.parameters(), lr, momentum)
criterion = torch.nn.BCELoss()              # Check for data imbalance if weight is needed

In [15]:
temp = dataset[0][0].clone().to(device)

In [17]:
store = model.forward(temp)

In [18]:
del(store)

In [18]:
store

tensor([[0.6752],
        [0.7288],
        [0.6271],
        [0.7372],
        [0.6037],
        [0.4924],
        [0.6377],
        [0.6962],
        [0.6178],
        [0.4839],
        [0.6776],
        [0.5733],
        [0.5997],
        [0.5323],
        [0.6604],
        [0.5117],
        [0.6650],
        [0.5396],
        [0.5680],
        [0.5624],
        [0.5228],
        [0.6980],
        [0.6853],
        [0.5211],
        [0.5718],
        [0.4978],
        [0.4817],
        [0.6788],
        [0.6544],
        [0.6269],
        [0.5570],
        [0.7462],
        [0.5467],
        [0.6230],
        [0.5593],
        [0.5484],
        [0.6570],
        [0.5644],
        [0.5614],
        [0.6412],
        [0.6092],
        [0.6294],
        [0.5272],
        [0.7506],
        [0.4764],
        [0.6761],
        [0.6091],
        [0.5942],
        [0.6198],
        [0.4625],
        [0.7501],
        [0.4322],
        [0.6099],
        [0.6341],
        [0.6706],
        [0